In [1]:

import sys
import yaml
import torch
import logging
from pathlib import Path

# Add the path to the directory containing the omnicell package
# Assuming the omnicell package is in the parent directory of your notebook
sys.path.append('..')  # Adjust this path as needed

import yaml
import torch
import logging
from pathlib import Path
from omnicell.config.config import Config, ETLConfig, ModelConfig, DatasplitConfig, EvalConfig, EmbeddingConfig
from omnicell.data.loader import DataLoader
from omnicell.constants import PERT_KEY, GENE_EMBEDDING_KEY, CONTROL_PERT
import scanpy as sc
from omnicell.evaluation.utils import get_DEGs
from omnicell.data.catalogue import Catalogue

In [2]:
dd = Catalogue.get_dataset_details('AdamsonWeissman2016_GSM2406675_10X001')

adata = sc.read(dd.path, backed="r+")

In [3]:
adata.obs

,perturbation,read count,UMI count,tissue_type,cell_line,cancer,disease,perturbation_type,celltype,organism,ncounts,ngenes,percent_mito,percent_ribo,nperts
cell_barcode,,,,,,,,,,,,,,,
AAACATACACCGAT,CREB1_pDS269,1286.0,98.0,cell_line,K562,True,chronic myelogenous leukemia,CRISPR,lymphoblasts,human,8138.0,2412,0.0,34.037846,2
AAACATACAGAGAT,SNAI1_pDS266,296.0,19.0,cell_line,K562,True,chronic myelogenous leukemia,CRISPR,lymphoblasts,human,8980.0,2386,0.0,40.011135,2
AAACATACCAGAAA,62(mod)_pBA581,1829.0,162.0,cell_line,K562,True,chronic myelogenous leukemia,CRISPR,lymphoblasts,human,28610.0,4404,0.0,40.003494,2
AAACATACGTTGAC,EP300_pDS268,1580.0,98.0,cell_line,K562,True,chronic myelogenous leukemia,CRISPR,lymphoblasts,human,11346.0,2815,0.0,35.184204,2
AAACATACTGTTCT,62(mod)_pBA581,748.0,51.0,cell_line,K562,True,chronic myelogenous leukemia,CRISPR,lymphoblasts,human,9864.0,2584,0.0,35.817112,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGACTGGAAGGC,SNAI1_pDS266,331.0,22.0,cell_line,K562,True,chronic myelogenous leukemia,CRISPR,lymphoblasts,human,5942.0,1856,0.0,31.605520,2
TTTGACTGGACGAG,SNAI1_pDS266,745.0,35.0,cell_line,K562,True,chronic myelogenous leukemia,CRISPR,lymphoblasts,human,8150.0,2263,0.0,36.797546,2
TTTGCATGCCCGTT,SNAI1_pDS266,784.0,37.0,cell_line,K562,True,chronic myelogenous leukemia,CRISPR,lymphoblasts,human,9179.0,2473,0.0,37.128227,2


In [5]:
adata.obs

,batch,gene,gene_id,transcript,gene_transcript,guide_id,percent_mito,UMI_count,z_gemgroup_UMI,core_scale_factor,...,perturbation,organism,perturbation_type,tissue_type,ncounts,ngenes,nperts,percent_ribo,celltype,cell_type
cell_barcode,,,,,,,,,,,,,,,,,,,,,
AAACCCAAGAAACTAC-53,53,MRPS31,ENSG00000102738,P1P2,5261_MRPS31_P1P2_ENSG00000102738,MRPS31_-_41345123.23-P1P2|MRPS31_+_41345107.23...,0.051790,38405.0,0.065533,2.985979,...,MRPS31,human,CRISPR,cell_line,37804.0,6085,1,0.190747,retinal pigment epithelial cells,rpe1
AAACCCAAGAAGCCAC-51,51,LRRC37A3,ENSG00000176809,P1P2,4661_LRRC37A3_P1P2_ENSG00000176809,LRRC37A3_+_62915581.23-P1P2|LRRC37A3_-_6291539...,0.048614,12774.0,1.087875,0.721063,...,LRRC37A3,human,CRISPR,cell_line,12593.0,3588,1,0.207893,retinal pigment epithelial cells,rpe1
AAACCCAAGAAGCGAA-32,32,SRCAP,ENSG00000080603,P1P2,8397_SRCAP_P1P2_ENSG00000080603,SRCAP_-_30710672.23-P1P2|SRCAP_-_30710513.23-P1P2,0.049437,15353.0,0.571514,0.989524,...,SRCAP,human,CRISPR,cell_line,15118.0,4225,1,0.176677,retinal pigment epithelial cells,rpe1
AAACCCAAGAATACAC-44,44,WBP1,ENSG00000239779,P1P2,9793_WBP1_P1P2_ENSG00000239779,WBP1_-_74685599.23-P1P2|WBP1_-_74685547.23-P1P2,0.059373,18729.0,0.281205,1.321505,...,WBP1,human,CRISPR,cell_line,18491.0,4129,1,0.205830,retinal pigment epithelial cells,rpe1
AAACCCAAGAATCGAT-43,43,RRP12,ENSG00000052749,P1P2,7561_RRP12_P1P2_ENSG00000052749,RRP12_+_99161057.23-P1P2|RRP12_-_99161036.23-P1P2,0.059395,11634.0,0.633576,0.735945,...,RRP12,human,CRISPR,cell_line,11467.0,3258,1,0.200488,retinal pigment epithelial cells,rpe1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTTGTCTGCACCT-44,44,MAX,ENSG00000125952,P1P2,4871_MAX_P1P2_ENSG00000125952,MAX_+_65569008.23-P1P2|MAX_-_65568906.23-P1P2,0.079629,22228.0,0.767826,1.321505,...,MAX,human,CRISPR,cell_line,21798.0,4803,1,0.154510,retinal pigment epithelial cells,rpe1
TTTGTTGTCTGGGCAC-32,32,ATP6V0C,ENSG00000185883,P1P2,675_ATP6V0C_P1P2_ENSG00000185883,ATP6V0C_+_2564168.23-P1P2|ATP6V0C_-_2563995.23...,0.049527,12377.0,-0.004215,0.989524,...,ATP6V0C,human,CRISPR,cell_line,12149.0,3798,1,0.147584,retinal pigment epithelial cells,rpe1
TTTGTTGTCTGTCCCA-37,37,MLST8,ENSG00000167965,P1P2,5115_MLST8_P1P2_ENSG00000167965,MLST8_+_2255502.23-P1P2|MLST8_-_2255521.23-P1P2,0.061164,8093.0,-0.616659,0.908722,...,MLST8,human,CRISPR,cell_line,7990.0,2810,1,0.203880,retinal pigment epithelial cells,rpe1


In [76]:
adata.obs

,batch,gene,gene_id,transcript,gene_transcript,guide_id,percent_mito,UMI_count,z_gemgroup_UMI,core_scale_factor,...,perturbation,organism,perturbation_type,tissue_type,ncounts,ngenes,nperts,percent_ribo,celltype,cell_type
cell_barcode,,,,,,,,,,,,,,,,,,,,,
AAACCCAAGAAACTAC-53,53,MRPS31,ENSG00000102738,P1P2,5261_MRPS31_P1P2_ENSG00000102738,MRPS31_-_41345123.23-P1P2|MRPS31_+_41345107.23...,0.051790,38405.0,0.065533,2.985979,...,MRPS31,human,CRISPR,cell_line,37804.0,6085,1,0.190747,retinal pigment epithelial cells,rpe1
AAACCCAAGAAGCCAC-51,51,LRRC37A3,ENSG00000176809,P1P2,4661_LRRC37A3_P1P2_ENSG00000176809,LRRC37A3_+_62915581.23-P1P2|LRRC37A3_-_6291539...,0.048614,12774.0,1.087875,0.721063,...,LRRC37A3,human,CRISPR,cell_line,12593.0,3588,1,0.207893,retinal pigment epithelial cells,rpe1
AAACCCAAGAAGCGAA-32,32,SRCAP,ENSG00000080603,P1P2,8397_SRCAP_P1P2_ENSG00000080603,SRCAP_-_30710672.23-P1P2|SRCAP_-_30710513.23-P1P2,0.049437,15353.0,0.571514,0.989524,...,SRCAP,human,CRISPR,cell_line,15118.0,4225,1,0.176677,retinal pigment epithelial cells,rpe1
AAACCCAAGAATACAC-44,44,WBP1,ENSG00000239779,P1P2,9793_WBP1_P1P2_ENSG00000239779,WBP1_-_74685599.23-P1P2|WBP1_-_74685547.23-P1P2,0.059373,18729.0,0.281205,1.321505,...,WBP1,human,CRISPR,cell_line,18491.0,4129,1,0.205830,retinal pigment epithelial cells,rpe1
AAACCCAAGAATCGAT-43,43,RRP12,ENSG00000052749,P1P2,7561_RRP12_P1P2_ENSG00000052749,RRP12_+_99161057.23-P1P2|RRP12_-_99161036.23-P1P2,0.059395,11634.0,0.633576,0.735945,...,RRP12,human,CRISPR,cell_line,11467.0,3258,1,0.200488,retinal pigment epithelial cells,rpe1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTTGTCTGCACCT-44,44,MAX,ENSG00000125952,P1P2,4871_MAX_P1P2_ENSG00000125952,MAX_+_65569008.23-P1P2|MAX_-_65568906.23-P1P2,0.079629,22228.0,0.767826,1.321505,...,MAX,human,CRISPR,cell_line,21798.0,4803,1,0.154510,retinal pigment epithelial cells,rpe1
TTTGTTGTCTGGGCAC-32,32,ATP6V0C,ENSG00000185883,P1P2,675_ATP6V0C_P1P2_ENSG00000185883,ATP6V0C_+_2564168.23-P1P2|ATP6V0C_-_2563995.23...,0.049527,12377.0,-0.004215,0.989524,...,ATP6V0C,human,CRISPR,cell_line,12149.0,3798,1,0.147584,retinal pigment epithelial cells,rpe1
TTTGTTGTCTGTCCCA-37,37,MLST8,ENSG00000167965,P1P2,5115_MLST8_P1P2_ENSG00000167965,MLST8_+_2255502.23-P1P2|MLST8_-_2255521.23-P1P2,0.061164,8093.0,-0.616659,0.908722,...,MLST8,human,CRISPR,cell_line,7990.0,2810,1,0.203880,retinal pigment epithelial cells,rpe1


In [77]:

adata_control  = adata[adata.obs[dd.pert_key] == CONTROL_PERT]
adata_pert = adata[adata.obs[dd.pert_key] != 'CDK6']
get_DEGs(control_adata=adata_control, target_adata=adata_pert)

/orcd/data/omarabu/001/opitcho/miniforge3/envs/omnicell/lib/python3.9/site-packages/scanpy/preprocessing/_utils.py:30: RuntimeWarning: Mean of empty slice.
  return X.mean(axis=axis, dtype=dtype)
/orcd/data/omarabu/001/opitcho/miniforge3/envs/omnicell/lib/python3.9/site-packages/numpy/core/_methods.py:121: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(


KeyboardInterrupt: 